This file is meant to take accidents_bp_distance_ta which is the old accidents<br>
And use this file i_m_h_tel_aviv_yafo.csv which has new accidents.

This file is changing accidents position using epsg=4326, which by my understanding is not the correct format used by למ"ס

In [33]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [65]:
BICYCLE = 15
SCOOTER = 21
E_BICYCLE = 23

micro_m = [SCOOTER, E_BICYCLE, BICYCLE]

## Loading old and new data

In [66]:
involved_markers_heb_ta_old = gpd.read_parquet('./csv_tables/accidents_bp_distance_ta.parquet')
involved_markers_heb_ta_old['accident_date'] = pd.to_datetime(involved_markers_heb_ta_old['accident_date'])
involved_markers_heb_ta_old.set_geometry('geometry', inplace=True)

involved_markers_heb_ta_old.shape

(8435, 167)

In [67]:
involved_markers_heb_ta_new = pd.read_csv('./csv_tables/i_m_h_tel_aviv_yafo.csv')
involved_markers_heb_ta_new.shape

C:\Users\itay8\AppData\Local\Temp\ipykernel_19288\1805868235.py:1: DtypeWarning: Columns (151) have mixed types. Specify dtype option on import or set low_memory=False.
  involved_markers_heb_ta_new = pd.read_csv('./csv_tables/i_m_h_tel_aviv_yafo.csv')


(108562, 161)

## Fixing Columns in involved_markers_heb_ta_new

In [37]:
involved_markers_heb_ta_new.shape, involved_markers_heb_ta_old.shape

((108562, 161), (8435, 167))

In [38]:
involved_markers_heb_ta_new.columns[~involved_markers_heb_ta_new.columns.isin(involved_markers_heb_ta_old.columns)]

Index(['vehicle_vehicle_type', 'vehicle_vehicle_type_hebrew'], dtype='object')

In [39]:
involved_markers_heb_ta_old.columns[~involved_markers_heb_ta_old.columns.isin( involved_markers_heb_ta_new.columns)]

Index(['accident_date', 'geometry', 'poly_geo', 'distance', 'index_right',
       'oid_shvil', 'street_name', 'shemmikta'],
      dtype='object')

In [ ]:
involved_markers_heb_ta_new.drop(columns=['vehicle_vehicle_type','vehicle_vehicle_type_hebrew'], inplace=True)

In [40]:
# adding missing column accident date:

involved_markers_heb_ta_new['accident_date'] = pd.to_datetime(
    involved_markers_heb_ta_new['accident_year'].astype(str) + '-' + 
    involved_markers_heb_ta_new['accident_month'].astype(str) + '-' + 
    involved_markers_heb_ta_new['accident_day'].astype(str) + ' ' + 
    involved_markers_heb_ta_new['accident_hour'].astype(str) + ':' + 
    involved_markers_heb_ta_new['accident_minute'].astype(str)
)



In [ ]:
# Checking again columns
involved_markers_heb_ta_new.columns[~involved_markers_heb_ta_new.columns.isin(involved_markers_heb_ta_old.columns)]

In [41]:
involved_markers_heb_ta_new.provider_and_id.isin(involved_markers_heb_ta_old.provider_and_id).sum()

16129

In [42]:
involved_markers_heb_ta_old.provider_and_id.isin(involved_markers_heb_ta_new.provider_and_id).sum()

8431

In [43]:
involved_markers_heb_ta_new.shape, involved_markers_heb_ta_old.shape

((108562, 162), (8435, 167))

Loading additional data

In [44]:
# Street names

clean_st_names = []
bp_street_names = pd.read_csv('./csv_tables/bp_street_name_only.csv', index_col=0)
for index, row in bp_street_names.iterrows():
    if isinstance(row['street_name'], float):
        clean_st_names.append('')
    else:
        st_name = row['street_name'].split("מ-")
        clean_st_names.append(st_name[0].strip())

clean_st_names

['נמיר',
 'פרופס',
 'יאיר רוזנבלום',
 'קרן קיימת לישראל',
 'רקנאטי בגינה',
 '2426',
 'שי עגנון',
 'איינשטיין',
 'איינשטיין',
 'שי עגנון',
 'משעול 2364',
 'בני אפריים',
 'אהרון בקר',
 'אהרון בקר',
 'בני אפרים',
 'בני אפרים',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון שביל 7.10',
 'פארק הירקון שביל 7.10',
 'פארק הירקון שביל 7.10',
 'פארק הירקון שביל 7.10',
 'פארק הירקון שביל 7.10',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'רוקח',
 'רוקח',
 'רוקח',
 'רוקח',
 'רוקח',
 'רוקח',
 'רוקח',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'פארק הירקון',
 'נמל ת"א',
 'חבקוק',
 '2397',
 'הרברט סמואל',
 'נורדאו',
 'נורדאו',
 'נורדאו',
 'פנקס'

In [45]:
# Load BP
bicycle_paths = gpd.read_file("./csv_tables/tel_avlv_bycicle_UTM/Bicycle Routes.shp")
bicycle_paths.columns = ['oid_shvil', 'msorech', 'dateimport', 'width', 'direction', 'create_year',
                         'shemmikta', 'miflas', 'create_date', 'geometry']
bicycle_paths.create_year = bicycle_paths.create_year.fillna(-1).astype(int)
bicycle_paths
# changing col names to fit other dataframes 
bicycle_paths.oid_shvil = pd.to_numeric(bicycle_paths.oid_shvil, downcast='integer')
bicycle_paths.crs

bicycle_paths['street_name'] = clean_st_names

bp = bicycle_paths.to_crs(epsg=4326)

In [46]:
# Load Meta Data
# Path to your Excel file
excel_file_path = "./csv_tables/tel_avlv_bycicle_UTM/export.xlsx"

# Load the Excel file
bicycle_paths_meta_data = pd.read_excel(excel_file_path, engine='openpyxl')

bicycle_paths_meta_data.bitzua = bicycle_paths_meta_data.bitzua.fillna(-1).astype(int)
# Display the first few rows of the dataframe
bicycle_paths_meta_data[~bicycle_paths_meta_data.date_created.isna()]


,oid_shvil,ms_orech,date_import,width,direction,bitzua,shem_mikta,miflas,date_created
4,5,362.67,11/02/2024 00:31:49,2.0,דו סטרי,2018,רקנאטי בגינה מקרן קיימת לישראל עד אחימאיר,NaN,30/06/2018
15,16,1853.03,11/02/2024 00:31:49,3.5,דו סטרי,2022,פארק הירקון שביל 7.10,מדרכה,01/01/2022
30,31,920.95,11/02/2024 00:31:49,1.6,NaN,2021,נמיר מהרב גורן עד דורי שביל 7,כביש,01/09/2021
38,39,200.08,11/02/2024 00:31:49,2.0,NaN,2021,מרמורק מזרח מאבן גבירול עד הוברמן (מחודש),כביש,29/04/2021
76,77,164.29,11/02/2024 00:31:49,2.4,דו סטרי,2021,קרליבך מ- 454 עד ביליס מנחם,כביש,01/12/2021
...,...,...,...,...,...,...,...,...,...
429,430,212.81,11/02/2024 00:31:49,2.5,דו סטרי,2023,גשר איינשטיין,כביש,30/10/2023
431,432,1083.02,11/02/2024 00:31:49,2.5,דו סטרי,2023,נמיר מפרופס עד איינשטיין,כביש,30/11/2023
439,440,2210.57,11/02/2024 00:31:49,2.5,חד סטרי,2023,אבן גבירול החדש/2040 מפרופס עד יוניצ'מן גדה מז...,כביש,01/12/2023
448,449,2168.04,11/02/2024 00:31:49,2.5,חד סטרי,2023,אבן גבירול החדש/2040 מפרופס עד יוניצמן גדה מערבית,כביש,01/12/2023


In [47]:
bicycle_paths_and_meta_data = pd.merge(bp, bicycle_paths_meta_data, how='right', on='oid_shvil')
bicycle_paths_and_meta_data.head()

,oid_shvil,msorech,dateimport,width_x,direction_x,create_year,shemmikta,miflas_x,create_date,geometry,street_name,ms_orech,date_import,width_y,direction_y,bitzua,shem_mikta,miflas_y,date_created
0,1,95.10,11/02/2024 00:31:49,2.5,דו סטרי,2012,נמיר מיוניצ'מן דרומה,None,None,"LINESTRING (34.80054 32.14000, 34.80089 32.140...",נמיר,95.10,11/02/2024 00:31:49,2.5,דו סטרי,2012,נמיר מיוניצ'מן דרומה,NaN,NaN
1,2,201.14,11/02/2024 00:31:49,2.5,דו סטרי,2013,פרופס מנמיר עד קדושי השואה,None,None,"LINESTRING (34.79394 32.12300, 34.79327 32.123...",פרופס,201.14,11/02/2024 00:31:49,2.5,דו סטרי,2013,פרופס מנמיר עד קדושי השואה,NaN,NaN
2,3,696.31,11/02/2024 00:31:49,2.1,דו סטרי,2004,יאיר רוזנבלום מאורי צבי גרינברג עד פרופס,None,None,"LINESTRING (34.79394 32.12300, 34.79413 32.123...",יאיר רוזנבלום,696.31,11/02/2024 00:31:49,2.1,דו סטרי,2004,יאיר רוזנבלום מאורי צבי גרינברג עד פרופס,NaN,NaN
3,4,659.20,11/02/2024 00:31:49,2.5,דו סטרי,2018,"קק""ל מנמיר עד חיים לבנון",מדרכה,None,"LINESTRING (34.79522 32.12243, 34.79581 32.122...",קרן קיימת לישראל,659.20,11/02/2024 00:31:49,2.5,דו סטרי,2018,"קק""ל מנמיר עד חיים לבנון",מדרכה,NaN
4,5,362.67,11/02/2024 00:31:49,2.0,דו סטרי,2018,רקנאטי בגינה מקרן קיימת לישראל עד אחימאיר,None,None,"LINESTRING (34.80148 32.12168, 34.80166 32.122...",רקנאטי בגינה,362.67,11/02/2024 00:31:49,2.0,דו סטרי,2018,רקנאטי בגינה מקרן קיימת לישראל עד אחימאיר,NaN,30/06/2018


In [ ]:
# Filtering by micro mobility
i_m_h_ta_micro_m = involved_markers_heb_ta_new[(involved_markers_heb_ta_new.involve_vehicle_type.isin(micro_m))  ]

## Creating geometry column for new data based after filtering by micro mobility

In [48]:
from shapely.geometry import Point

longitude = involved_markers_heb_ta_new[(involved_markers_heb_ta_new.involve_vehicle_type.isin(micro_m))].longitude.dropna()
latitude = involved_markers_heb_ta_new[(involved_markers_heb_ta_new.involve_vehicle_type.isin(micro_m))].latitude.dropna()
# Convert DataFrame to GeoDataFrame
accidents_gdf = gpd.GeoDataFrame(
    i_m_h_ta_micro_m,
    geometry=[Point(xy) for xy in zip(longitude, latitude)]
)

# Set CRS for accidents_gdf if it hasn't been set already
accidents_gdf.crs = "EPSG:4326"


In [49]:
accidents_gdf.set_geometry('geometry', inplace=True)


## Get the new rows that need to be calculated

In [61]:
# Find the 'provider_and_id' values that are in accidents_gdf_new but not in accidents_gdf
accidents_in_new_not_old = accidents_gdf.loc[~accidents_gdf['provider_and_id'].isin(involved_markers_heb_ta_old['provider_and_id'])].copy()

# Find the 'provider_and_id' values that are in accidents_gdf but not in accidents_gdf_new
accidents_in_old_not_in_new = involved_markers_heb_ta_old.loc[~involved_markers_heb_ta_old['provider_and_id'].isin(accidents_gdf['provider_and_id'])].copy()

# Print or further process the differences
print("Rows in accidents_gdf_new but not in accidents_gdf:")
print(accidents_in_new_not_old.shape)

print("\nRows in accidents_gdf but not in accidents_gdf_new:")
print(accidents_in_old_not_in_new.shape)

Rows in accidents_gdf_new but not in accidents_gdf:
(0, 165)

Rows in accidents_gdf but not in accidents_gdf_new:
(6, 167)


## Calculate the Closest LineString and Distance for new rows

In [62]:
import geopandas as gpd
from shapely.geometry import Point, LineString
from geopy.distance import geodesic
import numpy as np

# Function to calculate minimum geodesic distance between a point and a linestring
def min_geodesic_distance(point, line):
    min_distance = float('inf')  # Start with a very large number
    
    # Iterate through each segment of the line
    points_list = list(line.coords)
    for i in range(len(points_list) - 1):
        segment_start = Point(points_list[i])
        segment_end = Point(points_list[i+1])
        
        # Calculate distance to the start and end points of each segment
        start_distance = geodesic((point.y, point.x), (segment_start.y, segment_start.x)).meters
        end_distance = geodesic((point.y, point.x), (segment_end.y, segment_end.x)).meters
        
        # Keep track of the minimum distance
        min_distance = min(min_distance, start_distance, end_distance)
    
    return min_distance

# Initialize columns for closest geometry and distance
accidents_in_old_not_in_new['poly_geo'] = None
accidents_in_old_not_in_new['distance'] = np.inf

total_accidents = len(involved_markers_heb_ta_new)
count = 0

# Process each accident in the DataFrame
for idx, accident in accidents_in_old_not_in_new.iterrows():
    min_distance = float('inf')
    closest_geom = None

    # Compare each accident to all bicycle paths
    for _, path in bp.iterrows():
        distance = min_geodesic_distance(accident.geometry, path.geometry)
        if distance < min_distance:
            min_distance = distance
            closest_geom = path.geometry

    # Update GeoDataFrame with the closest geometry and the shortest distance
    involved_markers_heb_ta_new.at[idx, 'poly_geo'] = closest_geom
    involved_markers_heb_ta_new.at[idx, 'distance'] = min_distance

    # Increment counter and print progress every 100 accidents
    count += 1
    if count % 100 == 0:
        print(f"Processed {count} of {total_accidents} accidents. ({count / total_accidents * 100:.2f}% complete)")

# Ensure 'poly_geo' is properly set as a GeoSeries
accidents_in_old_not_in_new['poly_geo'] = gpd.GeoSeries(accidents_in_old_not_in_new['poly_geo'], crs=accidents_in_old_not_in_new.crs)


In [64]:
accidents_in_new_not_old

,accident_id,provider_and_id,provider_code,file_type_police,involved_type,involved_type_hebrew,license_acquiring_date,age_group,age_group_hebrew,sex,...,total_weight,total_weight_hebrew,vehicle_vehicle_type,vehicle_vehicle_type_hebrew,vehicle_damage,vehicle_damage_hebrew,accident_date,geometry,poly_geo,distance


In [ ]:
involved_markers_heb_ta_new.set_geometry('poly_geo', inplace=True)

accidents_gdf_merged = gpd.sjoin(involved_markers_heb_ta_new, bp[['geometry', 'oid_shvil', 'street_name', 'shemmikta']], how='inner', predicate='intersects', lsuffix='left', rsuffix='right')

accidents_gdf_merged

In [ ]:
# adding the rows 
accidents_bp_distance = pd.concat([involved_markers_heb_ta_old, involved_markers_heb_ta_new], ignore_index=True)

## Export to file

In [ ]:
accidents_bp_distance.to_parquet('./csv_tables/accidents_bp_distance_ta.parquet')

In [ ]:
# # Adding new rows to a parquet file for now, will finish later
# accidents_gdf_merged = accidents_gdf_merged.copy()
# accidents_gdf_merged.set_geometry('geometry', inplace=True)
# accidents_gdf_merged.to_parquet('./csv_tables/accidents_bp_new_additional_rows.parquet')
